In [1]:
# THIS NOTEBOOK TAKES RAW DATA FROM EXPERIMENTS AND OUTPUT A PROCESSED DATASET TO BE USED AS INPUT FOR STREAMLIT APPLICATION

In [35]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [36]:
# load dataset
df = pd.read_csv("data_raw/yearly_log copy.csv")

# keep important columns to be used in the prototype
df = df[['timestamp','project_name','run_id','duration','emissions','emissions_rate','cpu_power','ram_power',
         'cpu_energy','ram_energy','energy_consumed','country_name','country_iso_code','region','os','python_version',
         'cpu_count','cpu_model','longitude','latitude','ram_total_size']]



# THIS STEP ADD THE FAKE DATE COLUMN FOR EVERY PROJECT EVERY ROW AS IT WAS A COMPUTATION OF THE DAY
# Sort by 'project_name'
df.sort_values(by='project_name', inplace=True)
# Group by 'project_name' and add a 'date' column
df['date'] = df.groupby('project_name').cumcount().apply(lambda x: datetime(2022, 1, 1) + timedelta(days=x))
# Insert the "date" column at the second position (index 1)
df.insert(1, 'date', df.pop('date'))


# THIS STEP DROP USELESS ROWS IN ORDER TO HAVE 365 RUNS FOR EVERY PROJECT (1 YEAR OF DEPLOYMENT RUNS OF THE 3 MODELS)
# Convert the 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])
# Sort DataFrame by 'project_name' and 'date'
df.sort_values(by=['project_name', 'date'], inplace=True)
# Group by 'project_name' and keep only the first 365 rows in each group
df = df.groupby('project_name').head(365)
# Reset index if needed
df.reset_index(drop=True, inplace=True)
#drop timestamp and run_id columns
df = df.drop(columns=['timestamp','run_id'])
# Normalize column 'project_name'
df['project_name'] = df['project_name'].str.lower().str.replace(' ', '')


# THIS STEP TRANSFORM COLUMNS TO FLOAT NUMBERS
# List of columns to transform
columns_to_transform = ['duration', 'emissions', 'emissions_rate','cpu_power','ram_power','cpu_energy','ram_energy','energy_consumed']
# Transform the specified columns to float
df[columns_to_transform] = df[columns_to_transform].astype(float)


# THIS STEP MULTIPLY SCIENTIFIC NOTATION COLUMNS BY A FACTOR TO RENDER THEM IN GRAPHS
df['duration'] = df['duration'] * 100
df['emissions'] = df['emissions'] * 100000000
df['emissions_rate'] = df['emissions_rate'] * 1000000
df['cpu_energy'] = df['cpu_energy'] * 1000000
df['ram_energy'] = df['ram_energy'] * 1000000
df['energy_consumed'] = df['energy_consumed'] * 1000000

'''

# THIS STEP IS NEEDED NOT NOW BUT FOR SURE IN PRODUCTION PHASE, TO DELETE ROWS THAT HAV A DATE PREVIOUS 1 YEAR AGO

# Define the cutoff date (today - 1 year)
cutoff_date = datetime.now() - timedelta(days=365)
# Filter rows based on the condition
df = df[df['date'] >= cutoff_date]

'''

"\n\n# THIS STEP IS NEEDED NOT NOW BUT FOR SURE IN PRODUCTION PHASE, TO DELETE ROWS THAT HAV A DATE PREVIOUS 1 YEAR AGO\n\n# Define the cutoff date (today - 1 year)\ncutoff_date = datetime.now() - timedelta(days=365)\n# Filter rows based on the condition\ndf = df[df['date'] >= cutoff_date]\n\n"

In [38]:
df.head(3)

,date,project_name,duration,emissions,emissions_rate,cpu_power,ram_power,cpu_energy,ram_energy,energy_consumed,country_name,country_iso_code,region,os,python_version,cpu_count,cpu_model,longitude,latitude,ram_total_size
0,2022-01-01,logisticregressionmodel,0.82960128784178999162,0.05801208731863231882,0.06992767268907028322,5.00000000000000000000,6.00000000000000000000,0.01135402255588107562,0.01339832941691080845,0.02475235197279188754,Albania,ALB,tirana,macOS-14.1-arm64-arm-64bit,3.9.6,10,Apple M1 Pro,19.81840000000000046043,41.32529999999999859028,16.00000000000000000000
1,2022-01-02,logisticregressionmodel,0.83539485931395995877,0.05869349046548200111,0.07025838118477507888,5.00000000000000000000,6.00000000000000000000,0.01149972279866536343,0.01354336738586425608,0.02504309018452962471,Albania,ALB,tirana,macOS-14.1-arm64-arm-64bit,3.9.6,10,Apple M1 Pro,19.81840000000000046043,41.32529999999999859028,16.00000000000000000000
2,2022-01-03,logisticregressionmodel,1.00972652435302001628,0.07095890232589509294,0.07027536725487266989,5.00000000000000000000,6.00000000000000000000,0.01390145884619818813,0.01637498537699381626,0.03027644422319200612,Albania,ALB,tirana,macOS-14.1-arm64-arm-64bit,3.9.6,10,Apple M1 Pro,19.81840000000000046043,41.32529999999999859028,16.00000000000000000000


In [39]:
# THIS STEP CREATE 3 DATAFRAMES, ONE FOR EVERY PROJECT, WITH THE AGGREGATIONS NEEDED FOR THE DASHBOARD WILDCARDS
# Aggregate data per project_name
grouped_wildcard_df = df.groupby('project_name').agg({'duration': 'sum', 
                                             'emissions': 'sum',
                                             'emissions_rate': 'sum',
                                             'cpu_power': 'sum',
                                             'ram_power': 'sum',
                                             'cpu_energy': 'sum',
                                             'ram_energy': 'sum',
                                             'energy_consumed': 'sum',
                                             'country_name': 'first',
                                             'region': 'first',
                                             'os': 'first',
                                             'python_version': 'first',
                                             'cpu_count': 'first',
                                             'cpu_model': 'first',
                                             'ram_total_size': 'first',}).reset_index()

# Save each group in a separate variable and in the relative folder
for group_name, group_data in grouped_wildcard_df.groupby('project_name'):
    print(group_name)
    globals()[f"{group_name}_wildcard_df"] = group_data.reset_index(drop=True)
    csv_filename = f"{group_name}_wildcard_df.csv"
    group_data.to_csv(f"data_processed/single_project/{csv_filename}", index=False)

logisticregressionmodel
projectexcecuterdataload
xgboostmodel


In [40]:
# THIS STEP CREATE 3 DATAFRAMES, ONE FOR EVERY PROJECT, WITH THE COLUMNS NEEDED FOR THE GRAPHS (LINECHARTS, ETC...)
# Aggregate data per project_name
logisticregressionmodel_series_df = df[df['project_name']=='logisticregressionmodel'][['project_name','date','duration','emissions','emissions_rate','cpu_power','ram_power','cpu_energy','ram_energy','energy_consumed']]
projectexcecuterdataload_series_df = df[df['project_name']=='projectexcecuterdataload'][['project_name','date','duration','emissions','emissions_rate','cpu_power','ram_power','cpu_energy','ram_energy','energy_consumed']]
xgboostmodel_series_df = df[df['project_name']=='xgboostmodel'][['project_name','date','duration','emissions','emissions_rate','cpu_power','ram_power','cpu_energy','ram_energy','energy_consumed']]

logisticregressionmodel_series_df.to_csv(f"data_processed/single_project/logisticregressionmodel_series_df.csv", index=False)
projectexcecuterdataload_series_df.to_csv(f"data_processed/single_project/projectexcecuterdataload_series_df.csv", index=False)
xgboostmodel_series_df.to_csv(f"data_processed/single_project/xgboostmodel_series_df.csv", index=False)

In [41]:
# THIS STEP CREATE THE SERIES FOR GRAPH THE VARIABLES IN THE MAIN DASHBOARD OF ALL PROJECTS TOGETHER

# Group by 'date' and sum 'emission' and 'utilization'
grouped_all_series_df = df.groupby('date').agg({'duration': 'sum', 
                                     'emissions': 'sum',
                                     'emissions_rate': 'sum',
                                     'cpu_power': 'sum',
                                     'ram_power': 'sum',
                                     'cpu_energy': 'sum',
                                     'ram_energy': 'sum',
                                     'energy_consumed': 'sum'}).reset_index()
grouped_all_series_df.to_csv("data_processed/whole_project/grouped_all_series_df.csv", index=False)

In [42]:
# THIS STEP CREATE THE SINGLE ROW FOR ALL PROJECT SUMMING THE WILDCARD VALUES FOR THE WHOLE MAIN DASHBOARD
grouped_all_wildcard_df = df[['duration','emissions','emissions_rate','cpu_power','ram_power','cpu_energy','ram_energy','energy_consumed']]
grouped_all_wildcard_df = pd.DataFrame([grouped_all_wildcard_df.sum()])
grouped_all_wildcard_df.to_csv("data_processed/whole_project/grouped_all_wildcard_df.csv", index=False)